# Tutorial 1: Quick Start

## How to use this tutorial

  * Select "run all cells" on this notebook from the Run menu in Jupyter notebook or Jupyter
    lab. This step will produce intermediate data output and charts.
  * Some cells print out a url, which you can click on and bring up an interactive web UI to
    visualize the graph data.
  * In the unlikely event that the notebook becomes irresponsive, you can try "Restart
    Kernel" from the Kernel menu, then run individual cells one by one using `Shift+Enter`.
  * Some tutorials use local clusters consisting of multiple processes to mimic the effects
    of graph distribution over a remote cluster. By default, these local clusters
    automatically stop after idling for 15 minutes to conserve CPU and memory resources. You
    will need to rerun the entire notebook if your local cluster stopped due to inactivity.
  * Additional resources (video demos & blogs) are available at http://juliustech.co.
  * To report any issues, get help or request features, please raise an issue at
    https://github.com/JuliusTechCo/JuliusGraph/issues.

## 0. Introduction

Julius is an auto-scaling, low-code, and visual graph computing solution, that can build
sophisticated data and analytical pipelines with very little code. This tutorial is a quick
start guide on the basic concepts and programming API of Julius Graph Engine. You can find
introductory videos and blogs on graph computing at http://juliustech.co if you are new to
the subject.

Julius Graph Engine is implemented in [Julia](https://julialang.org/). Prior knowledge in
Julia is not required to follow this tutorial, as developer mainly interact with Julius
GraphEngine through a domain specific language (DSL): the Julius RulesDSL, which only uses a
very limited set of Julia syntax.  If you have programming experience in Python or Matlab,
you can follow this guide without much difficulties. We will highlight and explain some key
Julia specific syntax in this tutorial for Python/Matlab developers.

This tutorial is organized around three key concept in Julius Graph Engine: Atoms, Rules and
Runtime.

## 1. Atoms

`Atom` is an abstract Julia type, it is the most fundamental building block of Julius Graph
Engine, it stands for atomic operations. The name `Atom` is chosen because they are the
minimal unit of distribution and caching in Julius Graph Engine, atoms cannot be broken
apart further for those purposes. `Atom` interface enables Julius Graph Engine to access
functionalities implemented in existing software libraries. `Atom` can be implemented in all
major programming languages, such as Python, C/C++, Java, .Net, R and Julia etc. Existing
functions written in these languages can be easily wrapped up as an atom and be used in
Julius Graph.

There are several different sub types of Atoms for different use cases. We first introduce
`Datom`, which is the most generic `Atom` type that encapsulates any numerical or data
operation. The `Datom` interface only has a single method called ```fwddata!```. The
```...``` syntax in the function signature is a Julia specific syntax for catching varying
number of arguments.

```julia
fwddata!(self::Datom, xs...)
```

The `fwddata!` method can take any number of input data object in `xs` and **must return a
vector** of output objects. The return type of vector is chosen to be explicit that the
output could contain multiple objects.

The follow code snippets creates a `Datom` that computes the weighted sum of an input vector
of `DataFrame` objects. `DataFrame` is the popular choice for representing tabular data in
data science applications. Julia's `DataFrame` provides similar functionalities as the
Python's pandas Dataframe. A developer is free to choose other input/output types for
writing their own `Datoms`, the `xs` in the `fwddata!` can be any data type.

The following cells contain a complete implementation of the weighted sum `Datom`.

In [ ]:
using DataFrames, Base.CoreLogging
using GraphEngine: RuleDSL, GraphVM # RuleDSL package has all the APIs and type definitions of Julius Graph Engine
using GraphIO

# turn off information logging output, only show errors and warning
disable_logging(CoreLogging.Info)

# start a data server for the web UI
gss = Dict{String,RuleDSL.AbstractGraphState}()
port = GraphVM.drawdataport()
@async GraphVM.startresponder(gss, port)

In [ ]:
# this method needs to be extended when declaring new datoms
import GraphEngine.RuleDSL: fwddata!

struct WSumDF <: RuleDSL.Datom
    weights::Vector{Float64}
end

function fwddata!(self::WSumDF, dfs::DataFrame...)
    @assert length(dfs) == length(self.weights) && length(dfs) > 0 "invalid input size"
    sum = self.weights[1] .* dfs[1]

    for i=2:length(self.weights)
        sum .+= (self.weights[i] .* dfs[i])
    end

    return [sum] # must return a vector
end

Next, we show how to use `Datom` to compute the weighted sum of `DataFrame`s.

In [ ]:
using Random

ws = [1.0; 2.0; 3.0]
xlen = 10
xs = [rand(xlen), rand(xlen), rand(xlen)]

wsumd = WSumDF(ws)
dfs = [DataFrame(v=x) for x in xs] # create 3 data frames
ysd = RuleDSL.fwddata!(wsumd, dfs...)
first(ysd)

Julius Graph Engine offers a convenient macro `RuleDSL.@datom` for writing new datoms, such
that he same weighed sum datom can be implemented more compactly as below. The
`RuleDSL.@datom` macro translate the code below to a code block similar to the `WSumDF`
implementation above. The two versions are equivalent to each other in functionality.
Besides making the code more compact and easier to read and write, `RuleDSL.@datom` is also
future proof, in that the macro would automatically generate conversion code in case of
future Julius API changes. Therefore, it is strongly recommended to use the `RuleDSL.@datom`
macro to declare new `Datom` types.

In [ ]:
# `WSumDF2` is the name of `Datom`'s type
RuleDSL.@datom WSumDF2 begin
    weights::Vector{Float64} # the member of WSumDF2

    function fwddata!(dfs::DataFrame...) # self::WSumDF2 is automatically inserted as the first argument
        sum = zeros(nrow(dfs[1]))
        for (w, df) in zip(weights, dfs) # here weights is translated to self.weights, accessing the member
            sum .+= w .* df[!, 1]
        end
        return [DataFrame(; wsum=sum)] # return a vector
    end
end

wsumd2 = WSumDF2(ws)
ysd2 = RuleDSL.fwddata!(wsumd2, dfs...)

# Horizontally concatenate data frames for comparison
hcat(ysd..., ysd2...; makeunique=true)

We also define another `Datom` type `RandDF` that returns uniform random numbers in
a `Dataframe`, for later use. The `meanv` parameter specifies the mean of the uniform
random numbers.

In [ ]:
RuleDSL.@datom RandDF begin
    n::Int
    meanv::Float64

    function fwddata!()
        return [DataFrame(v=(rand(n) .- 0.5 .+ meanv))] # create a random vector
    end
end

## 2. Rules

The `Atom` and its subtype `Datom` are generic abstractions for atomic numerical and data
processing algorithms. However, just having atoms is not enough to build a working system or
application, we still need to connect these atoms in a meaningful way to create a system or
application. The RuleDSL is a high level domain specific language (DSL) exactly designed for
that purpose. RuleDSL specifiies precisely how atoms should be connected to one another to
form a computation DAG (directed acyclic graph) for the entire system or application.

The RuleDSL is a graph programming language, it has a very limited syntax for creating DAGs.
It doesn't support most constructs in a conventional programming languages, such as
variables, functions, branches or inheritance. RuleDSL only contains discreate declaration
of rules, therefore it is much easier to learn and use than the full-fledged programming
language. The `Atom` is an interface to traditional programming languages and libraries. The
combination of Rules and Atoms are extremely powerful and expressive, they offer the best
features from the traditional and graph programming. Together they can create systems of any
scale and complexity with a minimal amount of code.

### 2.1 RuleDSL Syntax

We use Fibonacci sequence as an example to describe the RuleDSL syntax. The following is a
Fibonacci like sequence using the `WSumDF` datom we just defined. This example is different
from the classic Fibonacci sequence in that its two initial terms are random vectors intead
of scalar 0 and 1; thus the recursive summation is applied to random vectors instead of
scalars.

In [ ]:
RuleDSL.@addrules seq begin
    fib(n::Int) = RuleDSL.Alias(fib(n, Val(n <= 1))) # binds the 2nd parameter for convenience, Alias is an atom that passes the outputs of other rules

    fib(f::Float64) = RuleDSL.Alias(fib(Int(floor(f)))) # adapting rule for a floating point parameter

    fib(n::Int, isend::Val{false}) = begin # defines the recursion of Fibonacci sequence
        WSumDF[[1.0, 1.0]](fib(n - 1, Val(n <= 2)), fib(n - 2, Val(n <= 3)))
    end

    fib(n::Int, isend::Val{true}) = RandDF[10, Float64(n)]() # using the RandDF atom defined earlier, random vector of length 10.
end

The `RuleDSL.@addrules` is a macro provided by Julius Graph Engine for writing rules using
the Rules DSL. The first argument `seq` is a user defined namespace to help organize related
rules, followed by a list of discrete rules between the `begin` and `end`. There are four
rules defined in the above declarations, the basic syntax of individual rules are:

```julia
rulename(arg1::Type1, arg2::Type2, ...) = AtomName[a1, a2, a3...](deprule1(args1...), deprule2(args2...), ...)
```

For example, in the rule:
```julia
fib(n::Int, isend::Val{false}) = WSumDF[[1.0, 1.0]](fib(n - 1, Val(n <= 2)), fib(n - 2, Val(n <= 3)))
```

* `fib` is the rule name, the full name of a rule includes its namespace, in this case it is
`seq.fib`
* `n` and `isend` are two parameters to this rule, with type `Int, Val{false}`, all rule
  parameter must have corresponding types in the rule declarations. The `Val{false}` is a
  Julia templatized type that enables value based pattern matching (matching the value
  `false` in this case).
* `WSumDF` is the atom name, which was defined earlier, the outer square bracket encloses
  the parameter to the atom's constructor
* `[1.; 1.]` in the square bracket is the parameter to the Datom `WSumDF`'s constructor,
  i.e., it is the weights parameter. The `WSumDF[[1.; 1.]]` in the rule declaration is
  directly translated to a call of`WSumDF([1.; 1.])` to create the `WSumDF` object at run
  time.
* the `fib(n-1, Val(n<=2)), fib(n-2, Val(n<=3))` inside the parenthesis is a list of
  dependent rules, they must map to existing rules, so that the Graph Engine can recursively
  expand the rules at run time to create a complete computational graph. A rule can refer to
  other rules (including itself) as dependencies. In this case, it defines the recursive sum
  of Fibonacci sequence.

Besides the Datom `WSumDF` we have just defined, these `seq` rules also uses the Atoms defined in
the `GraphEngine.RuleDSL` package, such as `RuleDSL.Alias`. Julius Graph Engine comes with a
rich set of pre-built Atoms that can be used by referring to their fully qualified Atom
names. The atom `RuleDSL.Alias` is a special atom that simply pass the results of the
dependent rules, it creates an additional name to improve readability. Alias atom is often
used to bind certain rule parameters to concrete values, as shown in the first `seq.fib`
rule above.

### 2.2 Polymorphism and Rule Matching

Rules DSL supports multiple dispatch with type and value based polymorphism, as a result it
can capture complex data or analytical logic. Multiple rules can have the same rule name
with different argument types, for example we have four rules of `seq.fib` with different
parameters above. At run time, the rule with the most precise match by type are invoked,
this process continues recursively through rule dependencies until all the dependent rules
are resolved. The final result of this rule matching and expansion process is a fully
constructed computation graph (a DAG), which is ready to be executed.

The value based polymorphism is supported using Julia's templatized type, such as
`Val{true}` or `Val{false}`. For example, in the first `seq.fib` rule above, if `n <= 1`,
the isend parameter of the dependent rules become an instance of the type `Val{true}`, which
would match to the last `seq.fib` rule governing the initial terms.

To fully understand this process, let's go through the creation of computational graph for
`seq.fib(3.2)` step by step.

In [ ]:
# override label display on the node to show additional rule parameters
import GraphEngine.RuleDSL: nodelabel
function nodelabel(::AbstractGraphState, ref::NodeRef)
    hdr = "$(ref.ns).$(ref.name)"
    ps = join(simplerepr.(ref.params), ", ")

    return "$hdr($ps)"
end

# create a concrete instance to a rule
ref = RuleDSL.@ref seq.fib(3.2)

# configuration object to pass common parameters to run time
config = RuleDSL.Config()

# run the computation according to the defined rules to create output in the Set
gs = GraphVM.createlocalgraph(config, RuleDSL.GenericData());
GraphVM.calcfwd!(gs, Set([ref]));

svg = GraphIO.postlocalgraph(gss, deepcopy(gs), port, true, key="fib");
display("image/svg+xml", svg)

In the above example, the steps to create the entire computation DAG includes:

1. The best matching rule for `seq.fib(3.2)` is
   `fib(f::Float64)=RuleDSL.Alias(fib(Int(floor(f))))`, which is aliased to `seq.fib(3)`
   because the parameter `Int(floor(f))` in the depedent rule evaluates to 3.
2. The best matching rule for `seq.fib(3)` is `fib(n::Int)=RuleDSL.Alias(fib(n,
   Val{n<=1}()))`, whose dependent rule is `seq.fib(3, Val{false}())` where `Val(false)` is
   an instance of templatized type `Val{false}`.
3. The best matching rule for `seq.fib(3, Val(false))` is  `fib(n::Int,
   isend::Val{false})=WSumDF[[1.; 1.]](fib(n-1, Val(n<=2)), fib(n-2, Val(n<=3)))`, with 2
   dependent rules: `seq.fib(2, Val(false))` and `seq.fib(1, Val(true))` .
4. The best matching rule for `seq.fib(2, Val(false))` is again  `fib(n::Int,
   isend::Val{false})=WSumDF[[1.; 1.]](fib(n-1, Val(n<=2)), fib(n-2, Val(n<=3))`, with two
   dependent rules `seq.fib(1, Val(true))`  and `seq.fib(0, Val(true))`
5. The best matching rule for `seq.fib(1, Val(true))` and `seq.fib(0, Val(true))` is
   `fib(n::Int, isend::Val{true})=RandDF[10, Float64(n)]()`, which is the initial term of
   the Fibonacci sequence that does not have any further dependencies. Now the graph
   expansion/creation is complete.

The above cell's output contains a url to a web UI, where a user can visualize the entire
calculation with all the intemediate results.

All the data are cached in the gs object, which is of type `GraphVM.GraphState` and
contains all the run time state of the DAG. Data can be retrieved using the `getdata`
method, which takes the hash value of the `RuleDSL.NodeRef` object. The hash value of
RuleDSL.NodeRef object is often used as a unique ID of the node. The relationship between
rules, NodeRef, and unique id is the following:

Rule declaration $ \xrightarrow{\text{bind rule parameters by RuleDSL.@ref}} $
`RuleDSL.NodeRef` $ \xrightarrow{\text{hash}} $ Unique Node ID

The `RuleDSL.@ref` is a convenient macro to create `RuleDSL.NodeRef` object, which
represents fully parameterized rules. For example the following cells shows two
`RuleDSL.NodeRef` objects can be created from the same rule but with different arguments
thus having different hash IDs. Both of their values can be retrieved by `getdata`.

In [ ]:
fib2 = RuleDSL.@ref seq.fib(2, Val{false}())
fib3 = RuleDSL.@ref seq.fib(3, Val{false}())

println(typeof(fib2) => hash(fib2))
println(typeof(fib3) => hash(fib3))

v2 = RuleDSL.getdata(gs, hash(fib2))
v3 = RuleDSL.getdata(gs, hash(fib3))

hcat(v2..., v3...; makeunique=true)

### 2.3 Dynamic Dependency

A rule's dependency can by dynamic, i.e., the number and type of dependencies can be
different according to the rule's parameters. For example the following cell defines a sum
of all the even number of terms in the Fibonacci sequence:

In [ ]:
RuleDSL.@addrules seq begin
    sumeven(n::Int) = begin
        WSumDF[fill(1.0, length(0:2:n))](RuleDSL.@ref(fib(i) for i in 0:2:n)...)
    end
end

As shown below, the `RuleDSL.@ref(fib(i) for i in 0:2:n)` create a vector of RuleDSL.NodeRef
object from the list comprehension inside. The `...` is a special syntax in rule dependency
that signals dynamic rule dependencies, it can follow any valid Julia expression or
functions that returns an instance or a vector of RuleDSL.NodeRef.

In [ ]:
RuleDSL.@ref seq.fib(5)

In [ ]:
RuleDSL.@ref(seq.fib(i) for i in 0:2:5)

The results of the dynamic dependency is shown below.

In [ ]:
# create a concrete instance to a rule
sumeven = RuleDSL.@ref seq.sumeven(5)

# run the computation according to the defined rules to create the desired output
gs = GraphVM.createlocalgraph(config, RuleDSL.GenericData());
GraphVM.calcfwd!(gs, Set([sumeven])); # we want to compute sumeven

svg = GraphIO.postlocalgraph(gss, gs, port, true; key="even");
display("image/svg+xml", svg)

### 2.4 Multi-line Rule

So far, all rules we have shown are single line declarations. Sometimes it is convenient to
run simple calculations when declaring the rule. To allow multi-line declarations we can
wrap up them within `begin ... end` , as it is shown in the following example of an
equivalent declaration of rule `seq.sumeven(n::Int)`. The rule can be easier to read by
using the local variables `dep, ws`:

In [ ]:
RuleDSL.@addrules seq begin
    sumeven2(n::Int) = begin
        deps = RuleDSL.@ref(fib(i) for i in 0:2:n)
        ws = fill(1.0, length(deps))
        WSumDF[ws](deps...)
    end
end

## 3. Runtime and GraphData

The combination of Rules and Atoms are very powerful and expressive, they can build any data
and analytical pipeline as a directed acyclic graph (DAG) regardless of its complexity,
usually with very few lines of code in RuleDSL. However, a system needs additional run time
configurations to function properly, even with all the business logic being fully specified.
Runtime configuration is an important aspect of a running system, which is the topic of this
section.

Runtime configuration usually include the following attributes:
* Distribution: whether a system is running on a local computer or a distributed computing
  environment
* Caching: should the intermediate results being cached, and how/where it is cached
* Adjoint Algorithmic Differentiation (AAD): whether the system execution includes AAD
* Batch or Streaming: whether the system running once in a batch mode, or running in a live
  mode with endless streaming data

In a traditional development environment, separate and duplicated codebases are created in
order to support different runtimes. For example, it is common in a bank to have separate
and dedicated systems for each combination of (runtime, applications), such as end of day
batch system for Macro trading, live intraday system for Equity e-trading, test/UAD
environment for XVA, etc. These specialized and dedicated systems do not share much common
software, hardware or configuration. As a results, the number of these specialized system
can quick multiple and drive up the overall complexity and support cost in a bank, and at
the same time hurting its consistency and reliability.

Julis GraphEngine provides a number of common runtime configurations out of box. Different
runtime environment can be created on-demand from a common set of Rules and Atoms defining
the business logic. This removes the needs for duplicated implementation of runtime
configurations for individual sytems, leading to significant reduction in support cost and
system complexity.

Julius Runtime are implemented as different `GraphData` types. Each `GraphData` type
implements a specific runtime configuration. In the previous sections, we have already seen
the `GenericData` in action, which is a derived type of `GraphData`. The following table
shows some of the most common `GraphData` configurations in Julius:

| GraphData Type        | Caching | Batch/Stream | support AAD | supported Atom type |
|:---------------------:|:-------:|:------------:|:-----------:|:-------------------:|
| `RuleDSL.GenericData` | yes     | batch        | no          | any                 |
| `RuleDSL.NumericalData` | yes   | batch        | yes         | `Quantom` only      |
| `RuleDSL.StreamData`  | no      | stream       | no          | any                 |

Any `GraphData` can be used in either local or distributed mode, we only work with local
runtime in this notebook, but subsequent tutorials will show the distributed setup. The
`NumericalData` is a `GraphData` specialized for numerical computation with AAD, which we
will cover in another tutorial.

Next, we will show how streaming use cases can be easily created using the `StreamData`.

### 3.1 Streaming

Streaming is a common runtime use case, e.g., for live intraday pricing and risk during
market open. With the `RuleDSL.StreamData` runtime, Julius can turn any batch system to a
streaming system with few lines of code change.

In the following few cells, we take the Fibnacci sequence as defined above, which operates
in the batch mode, and turn it into a streaming processing. Its initial inputs, the `fib(0)`
 and `fib(1)` terms becomes the streaming input of random vectors.

To show the effects of streaming, we create a new Datom that averages all the streamed
input values.

In [ ]:
# computes the running average of all the value being streamed
RuleDSL.@datom StreamAverage begin
    sum::DataFrame = DataFrame()
    cnt::Vector = [0]

    function fwddata!(x::DataFrame)
        if cnt[1] == 0
            append!(sum, x)
        else
            sum .+= x
        end

        cnt[1] += 1
        [ sum ./ cnt[1] ]
    end
end

Then we define a generic rule for computing stream average from any node. One of the most
powerful feature of Julius Rules DSL, is the high order rule, which is a rule declaration
that takes a `RuleDSL.NodeRef` object as parameter.  A high order rule can declare a generic
pattern that is applicable to any other rules, for example map/reduce or Monte carlo
simulation can be expressed as generic high order rules. We will cover high order rules in
more detail in a following map/reduce tutorial.

In the following high order rule for stream average, the `...` operator in the dependency
indicates dynamic dependency, which is just the input rule as defined by the `NodeRef`
parameter. The `seq.streamaverage` is therefore a generic operation that can be applied to
any other rule as represented in its argument `ref`, and it has a single dependency to the
node specified by the ref.

In [ ]:
RuleDSL.@addrules seq begin
    streamaverage(ref::RuleDSL.NodeRef) = StreamAverage(ref...)
end

# sources of streaming inputs
srcs = [
    RuleDSL.@ref(seq.fib(0, Val{true}())), RuleDSL.@ref(seq.fib(1, Val{true}()))
]
sd = RuleDSL.StreamData(Set(hash.(srcs)), 1) # create StreamData with queue buffer length of 1

ref = RuleDSL.@ref seq.fib(5)
savg = RuleDSL.@ref seq.streamaverage(ref)

gs2 = GraphVM.createlocalgraph(config, sd) # create a graph using StreamData
GraphVM.calcfwd!(gs2, Set{NodeRef}(savg)); # set up the pipeline of streaming

The calcfwd! call above builts the pipeline between nodes for streaming. The connection
between nodes in StreamData are message queues, where each node runs asynchronously to
process the message from the input queue, and then puts the results into the output queue.

The following cell streams 20,000 different initial inputs, which are randomly drawn by the
datom `RandDF`. Since the final output's atom is StreamAverage, which computes the running
average of all the streamed data, the net result is essentially to compute the Fibonnaci
sequence using a Monte Carlo simulation of 20,000 paths, and the results are all close to 5,
which is the correct value of the 5th term of the classic Fibonacci sequence.

Unlike the mini-batch streaming processing in Spark, the streaming implementation in Julius
is fully pipelined, where different nodes are processing different streaming inputs
concurrently at any given time. As a result, Julius streaming is extremely fast, achieving
more than 10,000 messages per seconds.

In [ ]:
@time RuleDSL.pushpullcalc!(gs2, 10000) # stream 10,000 messages, this call can be made multiple times to stream more data
@time RuleDSL.pushpullcalc!(gs2, 10000)
avg = RuleDSL.getdata(gs2, hash(savg))[1] # [1] is because the output is a vector of 1

Once we are done with streaming, we can tear down the streaming pipeline by calling
`stopstream!` on `StreamData` to reclaim the resources. Once `stopstream!` is called, we can
no longer call the `pushpullcalc!`.

In [ ]:
RuleDSL.stopstream!(sd);

In [ ]:
GraphVM.servedata(gss, gs2, port; key="stream");

## 4. Exercise

Thanks for your attention! Now you have learnt the basics of **Julius Graph Engine**, it is
time to get your hands dirty and write your own rules and atoms! Here is an suggested
exercise:

Modify the streaming use case above, to also report the calculation of Monte Carlo error,
which is the standard deviation of the samples divided by the square root of the number of
samples. You can take advantage of the following relationship $\sigma^2(x)=E[x^2] - E^2[x]$
by tracking the average of squares. Then the MC error is $\frac{\sigma(x)}{\sqrt{n}}$ where
$n$ is the number of samples streamed.

There can be different approaches for doing this:
1. create a new Datom and corresponding rule, similar to `StreamAverage` and
   `seq.streamaverage`, that computes the running mean and MC error, and return them as two
   DataFrames in the output vector, or two columns in the same DataFrrame.
2. create new Datoms for arithmetic operations like Square, Subtraction and Sqrt on type
   DataFrame, then compute the MC error by writing new rules that connects these new datoms

The first approach is quick and fast, while the second approach is more generic and
reusable, with additional benefits of being able to see the intermediate results.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*